In [3]:
# create list
file2 = "common_nodes_hPPIN_STRING_PANCAN.txt"
combined_list = []
with open(file2) as IN:
    for line in IN:
        combined_list.append(line.rstrip().split("\t")[0])

In [1]:
# using ast.literal_eval()
import sys
import ast
import os
import time
import pandas as pd
import pickle
import glob
import numpy as np
import statistics
from scipy.stats import mannwhitneyu

In [2]:
with open("STRING_paths_no_scores.dictionary", 'rb') as config_dictionary_file:
 
    # Step 3
    paths = pickle.load(config_dictionary_file)

In [3]:
def compute_sp(list1, list2, paths):
    total_sp = []
    for i in range(len(list1)):
        for j in range(len(list2)):
            if(list1[i] in paths.keys()):
                shortest_paths = paths[list1[i]]
                if(list2[j] in shortest_paths.keys()):
                    total_sp.append(shortest_paths[list2[j]])
    return({"distribution": total_sp,
            "mean": statistics.mean(total_sp),
            "median": statistics.median(total_sp)})

In [4]:
# Read Cancersea Data
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/cancersea/"

filenames = os.listdir(folder_path)

cancersea = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    cancersea[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            cancersea[filename[:-4]].append(line.rstrip().split("\t")[1])

In [5]:
cancersea["COSMIC"] = list()
with open("/data/timonaj/cancer_as_wound/ppi_analysis/cosmic_hallmarks.txt") as IN:
    for line in IN:
        cancersea["COSMIC"].append(line.rstrip().split("\t")[0])

In [6]:
# read in HP sets
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets/"

filenames = os.listdir(folder_path)

hp_genesets = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    hp_genesets[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            hp_genesets[filename[:-4]].append(line.rstrip().split("\t")[0])

In [8]:
# read in HP matched controls set
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/"

filenames = glob.glob("/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/NS_STRING*")
hp_genesets_matched = {}

for filename in filenames:
    name = filename.split("NS_STRING")[1][:-4]
    hp_genesets_matched[name] = list()

    with open(filename) as IN:
        for line in IN:
            hp_genesets_matched[name].append(line.rstrip().split("\t")[0])

In [9]:
a_mean = np.zeros(shape=(len(hp_genesets), len(cancersea)))
a_median = np.zeros(shape=(len(hp_genesets), len(cancersea)))
a_dist = pd.DataFrame(index = range(len(cancersea)),
                      columns = range(len(hp_genesets)))

In [10]:
i=0
for key1,value1 in hp_genesets.items():
    j=0
    for key2,value2 in cancersea.items():
        current_info = compute_sp(value1, value2, paths)
        a_mean[i][j] = current_info["mean"]
        a_median[i][j] = current_info["median"]
        a_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [11]:
pd.DataFrame(a_median)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
5,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
6,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
7,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [12]:
pd.DataFrame(a_mean)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.829355,1.906462,1.826485,1.805040,1.766744,1.882681,1.846547,1.892224,1.800711,1.859265,1.869950,1.843480,1.913571,1.812915,1.876206
1,1.886088,1.913119,1.896747,1.879292,1.858294,1.901043,1.919275,1.931304,1.904615,1.913972,1.906058,1.891055,1.917783,1.887739,1.905815
2,1.857770,1.890710,1.823254,1.838052,1.795773,1.890503,1.889822,1.919382,1.861112,1.856359,1.887528,1.871178,1.907011,1.830938,1.893855
3,1.770990,1.932719,1.780932,1.753751,1.691427,1.873509,1.828260,1.880200,1.766082,1.835437,1.828131,1.819202,1.921688,1.726159,1.839376
4,1.746550,1.878139,1.751476,1.726567,1.675912,1.822222,1.838766,1.853621,1.788991,1.818797,1.792338,1.796610,1.874318,1.715604,1.808712
5,1.865057,1.900538,1.855841,1.845426,1.808181,1.895109,1.887063,1.919112,1.857305,1.880368,1.895321,1.867721,1.915629,1.847926,1.897597
6,1.847777,1.923358,1.851117,1.837492,1.802754,1.891187,1.882028,1.910778,1.850504,1.876114,1.881842,1.853997,1.920369,1.838091,1.887634
7,1.817014,1.871654,1.846633,1.792595,1.775717,1.822182,1.866256,1.863074,1.828702,1.873434,1.860625,1.842041,1.853177,1.814131,1.846122
8,1.905601,1.938897,1.916686,1.898811,1.880295,1.919367,1.932197,1.942108,1.917609,1.923593,1.923075,1.896509,1.931133,1.907761,1.922818


In [13]:
pd.DataFrame(a_mean, columns = cancersea.keys(), index = hp_genesets.keys()).to_csv("STRING_NS_meanSP_HP_Phenotypes.csv")

In [14]:
a_matched_mean = np.zeros(shape=(len(hp_genesets_matched), len(cancersea)))
a_matched_median = np.zeros(shape=(len(hp_genesets_matched), len(cancersea)))
a_matched_dist = pd.DataFrame(index = range(len(cancersea)),
                              columns = range(len(hp_genesets_matched)))

In [15]:
i=0
for key1,value1 in hp_genesets_matched.items():
    j=0
    for key2,value2 in cancersea.items():
        current_info = compute_sp(value1, value2, paths)
        a_matched_mean[i][j] = current_info["mean"]
        a_matched_median[i][j] = current_info["median"]
        a_matched_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [16]:
pd.DataFrame(a_matched_median)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
6,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
7,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
8,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [17]:
pd.DataFrame(a_matched_mean)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2.889065,2.882435,2.930672,2.868956,2.902643,2.853495,2.941080,2.941651,2.922311,2.916445,2.910638,2.925401,2.910025,2.907102,2.895551
1,2.959468,2.895954,2.972450,2.933873,2.967369,2.889857,2.980704,2.967557,2.976468,2.944974,2.960805,2.959127,2.951403,2.963874,2.960516
2,2.924539,2.889647,2.954303,2.902770,2.938187,2.872095,2.965347,2.958371,2.955811,2.933658,2.935232,2.945017,2.930983,2.937050,2.925789
3,2.879342,2.880450,2.924792,2.860071,2.893637,2.848652,2.935755,2.938261,2.915225,2.912421,2.903226,2.920803,2.904369,2.899219,2.886505
4,2.885492,2.881872,2.929346,2.866001,2.900261,2.851736,2.940394,2.941465,2.921476,2.915871,2.907882,2.924497,2.907959,2.904635,2.891486
5,2.868151,2.878389,2.918416,2.849806,2.883623,2.842812,2.929744,2.934359,2.906948,2.908160,2.895420,2.915631,2.897774,2.890326,2.876006
6,2.899327,2.884561,2.937764,2.878847,2.913216,2.858904,2.948527,2.946851,2.932619,2.921706,2.917685,2.931335,2.916099,2.915899,2.904050
7,2.882581,2.881008,2.926220,2.862830,2.896047,2.850249,2.936715,2.938710,2.916449,2.913185,2.905726,2.921832,2.906244,2.901572,2.889993
8,2.886530,2.882160,2.930533,2.867203,2.901863,2.852333,2.941929,2.942658,2.923648,2.916919,2.908470,2.925574,2.908593,2.905768,2.891891


In [18]:
pd.DataFrame(a_matched_mean, columns = cancersea.keys(), index = hp_genesets_matched.keys()).to_csv("STRING_NS_meanSP_HPmatched_Phenotypes.csv")

In [19]:
mannwhitneyu_p = np.zeros(shape=(len(hp_genesets), len(cancersea)))
test_dist = np.zeros(shape=(len(hp_genesets), len(cancersea)))
test_matched_dist = np.zeros(shape=(len(hp_genesets), len(cancersea)))

In [20]:
for i in range(len(hp_genesets)):
    for j in range(len(cancersea)):
        actual_dist = a_dist[i][j]
        control_dist = a_matched_dist[i][j]
        res = mannwhitneyu(actual_dist, control_dist, alternative="less")
        mannwhitneyu_p[i][j] = res.pvalue
        test_dist[i][j] = statistics.mean(actual_dist)
        test_matched_dist[i][j] = statistics.mean(control_dist)

In [21]:
pvals = pd.DataFrame(mannwhitneyu_p, columns = cancersea.keys(), index = hp_genesets.keys())

In [22]:
pvals

,Differentiation,Cell_Cycle,EMT,Proliferation,Metastasis,DNA_damage,Quiescence,Apoptosis,Inflammation,Invasion,Stemness,Hypoxia,DNA_repair,Angiogenesis,COSMIC
downregulated.wound,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
upregulated.stress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
downregulated.regen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
msigdb.wound,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
msigdb.regen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
upregulated.regen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
upregulated.wound,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
msigdb.stress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
downregulated.stress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
pvals.to_csv("STRING_NS_matched_pvals.csv")

In [24]:
pd.DataFrame(test_dist)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.829355,1.906462,1.826485,1.805040,1.766744,1.882681,1.846547,1.892224,1.800711,1.859265,1.869950,1.843480,1.913571,1.812915,1.876206
1,1.886088,1.913119,1.896747,1.879292,1.858294,1.901043,1.919275,1.931304,1.904615,1.913972,1.906058,1.891055,1.917783,1.887739,1.905815
2,1.857770,1.890710,1.823254,1.838052,1.795773,1.890503,1.889822,1.919382,1.861112,1.856359,1.887528,1.871178,1.907011,1.830938,1.893855
3,1.770990,1.932719,1.780932,1.753751,1.691427,1.873509,1.828260,1.880200,1.766082,1.835437,1.828131,1.819202,1.921688,1.726159,1.839376
4,1.746550,1.878139,1.751476,1.726567,1.675912,1.822222,1.838766,1.853621,1.788991,1.818797,1.792338,1.796610,1.874318,1.715604,1.808712
5,1.865057,1.900538,1.855841,1.845426,1.808181,1.895109,1.887063,1.919112,1.857305,1.880368,1.895321,1.867721,1.915629,1.847926,1.897597
6,1.847777,1.923358,1.851117,1.837492,1.802754,1.891187,1.882028,1.910778,1.850504,1.876114,1.881842,1.853997,1.920369,1.838091,1.887634
7,1.817014,1.871654,1.846633,1.792595,1.775717,1.822182,1.866256,1.863074,1.828702,1.873434,1.860625,1.842041,1.853177,1.814131,1.846122
8,1.905601,1.938897,1.916686,1.898811,1.880295,1.919367,1.932197,1.942108,1.917609,1.923593,1.923075,1.896509,1.931133,1.907761,1.922818


In [25]:
pd.DataFrame(test_matched_dist)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2.889065,2.882435,2.930672,2.868956,2.902643,2.853495,2.941080,2.941651,2.922311,2.916445,2.910638,2.925401,2.910025,2.907102,2.895551
1,2.959468,2.895954,2.972450,2.933873,2.967369,2.889857,2.980704,2.967557,2.976468,2.944974,2.960805,2.959127,2.951403,2.963874,2.960516
2,2.924539,2.889647,2.954303,2.902770,2.938187,2.872095,2.965347,2.958371,2.955811,2.933658,2.935232,2.945017,2.930983,2.937050,2.925789
3,2.879342,2.880450,2.924792,2.860071,2.893637,2.848652,2.935755,2.938261,2.915225,2.912421,2.903226,2.920803,2.904369,2.899219,2.886505
4,2.885492,2.881872,2.929346,2.866001,2.900261,2.851736,2.940394,2.941465,2.921476,2.915871,2.907882,2.924497,2.907959,2.904635,2.891486
5,2.868151,2.878389,2.918416,2.849806,2.883623,2.842812,2.929744,2.934359,2.906948,2.908160,2.895420,2.915631,2.897774,2.890326,2.876006
6,2.899327,2.884561,2.937764,2.878847,2.913216,2.858904,2.948527,2.946851,2.932619,2.921706,2.917685,2.931335,2.916099,2.915899,2.904050
7,2.882581,2.881008,2.926220,2.862830,2.896047,2.850249,2.936715,2.938710,2.916449,2.913185,2.905726,2.921832,2.906244,2.901572,2.889993
8,2.886530,2.882160,2.930533,2.867203,2.901863,2.852333,2.941929,2.942658,2.923648,2.916919,2.908470,2.925574,2.908593,2.905768,2.891891
